In [ ]:
#Install libraries
!pip install --quiet flask
!pip install --quiet --upgrade diffusers transformers accelerate mediapy triton scipy ftfy spacy==3.4.4
!pip install --quiet -q xformers==0.0.16rc425 
!pip install --quiet flask-ngrok

In [ ]:
#Set up Ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz

!./ngrok authtoken replace_with_user_token

In [ ]:
#import libraries
from flask import Flask, render_template, request
import requests
import base64
import numpy as np
from flask import Flask
from flask_ngrok import run_with_ngrok
from diffusers import StableDiffusionPipeline, DiffusionPipeline
import torch
from torch import autocast
import random
import io

In [ ]:
#set up stable diffusion model

model_id = "stabilityai/stable-diffusion-2-1-base"

scheduler = None

device = "cuda"

pipe = DiffusionPipeline.from_pretrained(model_id)

pipe = pipe.to(device)
pipe.enable_xformers_memory_efficient_attention()

In [ ]:
#Flask setup
app = Flask(__name__)
run_with_ngrok(app) 

In [ ]:
# Home page
@app.route('/')
def home():
    return render_template('index.html')

# Handle form submission
@app.route('/generate_image', methods=['POST'])
def generate_image():
    prompt_input = request.form['prompt_text']  
    negative_prompt_input = request.form['negative_prompt_text']
    image_length_input = request.form['image_length']
    image_width_output = request.form['image_width']

    generated_image = generate_image_from_text(prompt_input, negative_prompt_input,image_length_input,image_width_output)

    image_data = base64.b64encode(generated_image).decode('utf-8')

    return render_template('index.html', image_data=image_data, prompt_data = prompt_input, negative_prompt_data = negative_prompt_input)

def generate_image_from_text(prompt_input, negative_prompt_input = "", length = 512, width = 512):
    prompt = prompt_input
    negative_prompt = negative_prompt_input
    remove_safety = False
    image_length = length
    image_width = width
    num_images = 1
    seed = random.randint(0, 2147483647)

    images = pipe(
        prompt,
        height = image_length,
        width = image_length,
        num_inference_steps = 25,
        guidance_scale = 9,
        num_images_per_prompt = num_images,
        negative_prompt = negative_prompt,
        generator = torch.Generator("cuda").manual_seed(seed)
        ).images

    image_bytes = io.BytesIO()
    images[0].save(image_bytes, format='PNG')
    image_bytes.seek(0)

    return image_bytes.getvalue()


In [ ]:
#start flask web app on ngrok
if __name__ == '__main__':
    app.run()